#  Unit 2.4a Using Programs with Data, SQLAlchemy
> Using Programs with Data is focused on SQL and database actions.  Part A focuses on SQLAlchemy and an OOP programming style,
- toc: true
- week: 26

# Database and SQLAlchemy
> In this blog we will explore using programs with data, focused on Databases.  We will use SQLite Database to learn more about using Programs with Data.  ***Use Debugging through these examples to examine Objects created in Code***.

- College Board talks about ideas like 
    - Program Usage. "iterative and interactive way when processing information"
    - Managing Data.  "classifying data are part of the process in using programs", "data files in a Table"
    - Insight "insight and knowledge can be obtained from ...  digitally represented information"
    - Filter systems. 'tools for finding information and recognizing patterns"
    - Application. "the preserve has two databases", "an employee wants to count the number of book"

- PBL, Databases, Iterative/OOP
    - Iterative. Refers to a sequence of instructions or code being repeated until a specific end result is achieved
    - OOP. A computer programming model that organizes software design around data, or objects, rather than functions and logic
    - SQL. Structured Query Language, abbreviated as SQL, is a language used in programming, managing, and structuring data 


## Imports and Flask Objects
> Defines and key object creations

- Comment on where you have observed these working?  Provide a definition of purpose.
    1. Flask app object
        - Seen this used in past projects in trimester 1 and 2
        -  the purpose of the app object is to represent your web application. It is an instance of the Flask class, and it is used to configure your application and define its behavior. The app object is used to register routes, set up configuration variables, and define extensions that your application may use.
    2. SQLAlchemy db object
        - I have observed this working in my own CPT project. I use a sqlite database and I use the db object to read and create the database.
        - the purpose of the SQLAlchemy db object is to interact with your application's database. It is an instance of the SQLAlchemy class, and it provides a high-level interface for interacting with databases using Python code.


In [15]:
"""
These imports define the key objects
"""

from flask import Flask
from flask_sqlalchemy import SQLAlchemy

"""
These object and definitions are used throughout the Jupyter Notebook.
"""

# Setup of key Flask object (app)
app = Flask(__name__)
# Setup SQLAlchemy object and properties for the database (db)
database = 'sqlite:///instance/sqlite.db'  # path and filename of database
#database = 'sqlite:///files/sqlite.db'  # path and filename of database
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy()


# This belongs in place where it runs once per project
db.init_app(app)


## Model Definition
> Define columns, initialization, and CRUD methods for users table in sqlite.db

- Comment on these items in the class, purpose and definition.
    - class User: define the properties of the user including name, dob, password, etc.
    - db.Model inheritance: he ability to create a new model class that inherits from an existing model class, thereby inheriting all of its fields and methods
    - _init_ method: constructor of the user objects
    - ```@property```, ```@<column>.setter```: these are the getter and setter methods respectively
    - create, read, update, delete methods: these methods allow you to manipulate and access the data in the sqlite.db. Read allows you to view the data, create builds the data into the table, and update and delete do as the name states.

In [16]:
""" database dependencies to support sqlite examples """
import datetime
from datetime import datetime
import json

from sqlalchemy.exc import IntegrityError
from werkzeug.security import generate_password_hash, check_password_hash


''' Tutorial: https://www.sqlalchemy.org/library.html#tutorials, try to get into a Python shell and follow along '''

# Define the User class to manage actions in the 'users' table
# -- Object Relational Mapping (ORM) is the key concept of SQLAlchemy
# -- a.) db.Model is like an inner layer of the onion in ORM
# -- b.) User represents data we want to store, something that is built on db.Model
# -- c.) SQLAlchemy ORM is layer on top of SQLAlchemy Core, then SQLAlchemy engine, SQL
class User(db.Model):
    __tablename__ = 'users'  # table name is plural, class name is singular

    # Define the User schema with "vars" from object
    id = db.Column(db.Integer, primary_key=True)
    _name = db.Column(db.String(255), unique=False, nullable=False)
    _uid = db.Column(db.String(255), unique=True, nullable=False)
    _password = db.Column(db.String(255), unique=False, nullable=False)
    _dob = db.Column(db.Date)

    # constructor of a User object, initializes the instance variables within object (self)
    def __init__(self, name, uid, password="123qwerty", dob=datetime.today()):
        self._name = name    # variables with self prefix become part of the object, 
        self._uid = uid
        self.set_password(password)
        if isinstance(dob, str):  # not a date type     
            dob = date=datetime.today()
        self._dob = dob

    # a name getter method, extracts name from object
    @property
    def name(self):
        return self._name
    
    # a setter function, allows name to be updated after initial object creation
    @name.setter
    def name(self, name):
        self._name = name
    
    # a getter method, extracts uid from object
    @property
    def uid(self):
        return self._uid
    
    # a setter function, allows uid to be updated after initial object creation
    @uid.setter
    def uid(self, uid):
        self._uid = uid
        
    # check if uid parameter matches user id in object, return boolean
    def is_uid(self, uid):
        return self._uid == uid
    
    @property
    def password(self):
        return self._password[0:10] + "..." # because of security only show 1st characters

    # update password, this is conventional method used for setter
    def set_password(self, password):
        """Create a hashed password."""
        self._password = generate_password_hash(password, method='sha256')

    # check password parameter against stored/encrypted password
    def is_password(self, password):
        """Check against hashed password."""
        result = check_password_hash(self._password, password)
        return result
    
    # dob property is returned as string, a string represents date outside object
    @property
    def dob(self):
        dob_string = self._dob.strftime('%m-%d-%Y')
        return dob_string
    
    # dob setter, verifies date type before it is set or default to today
    @dob.setter
    def dob(self, dob):
        if isinstance(dob, str):  # not a date type     
            dob = date=datetime.today()
        self._dob = dob
    
    # age is calculated field, age is returned according to date of birth
    @property
    def age(self):
        today = datetime.today()
        return today.year - self._dob.year - ((today.month, today.day) < (self._dob.month, self._dob.day))
    
    # output content using str(object) is in human readable form
    # output content using json dumps, this is ready for API response
    def __str__(self):
        return json.dumps(self.read())

    # CRUD create/add a new record to the table
    # returns self or None on error
    def create(self):
        try:
            # creates a person object from User(db.Model) class, passes initializers
            db.session.add(self)  # add prepares to persist person object to Users table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            db.session.remove()
            return None

    # CRUD read converts self to dictionary
    # returns dictionary
    def read(self):
        return {
            "id": self.id,
            "name": self.name,
            "uid": self.uid,
            "dob": self.dob,
            "age": self.age,
        }

    # CRUD update: updates user name, password, phone
    # returns self
    def update(self, name="", uid="", password=""):
        """only updates values with length"""
        if len(name) > 0:
            self.name = name
        if len(uid) > 0:
            self.uid = uid
        if len(password) > 0:
            self.set_password(password)
        db.session.add(self) # performs update when id exists\n",
        db.session.commit()
        return self

    # CRUD delete: remove self
    # None
    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None
    

### My own Database Model: Cars

In [17]:
class Car(db.Model):
    __tablename__ = 'cars'  # table name is plural, class name is singular

    # Define the User schema with "vars" from object
    id = db.Column(db.Integer, unique=True, primary_key=True)
    _uid = db.Column(db.String(255), unique=True, nullable=False)
    _brand = db.Column(db.String(255), unique=False, nullable=False)
    _color = db.Column(db.Integer, unique=False, nullable=False)
    _powersource = db.Column(db.String(255), unique=False, nullable=False)
    _type = db.Column(db.String(255), unique=False, nullable=False)
   

    # constructor of a User object, initializes the instance variables within object (self)
    def __init__(self, uid, brand, color, type, powersource):
        self._brand = brand
        self._color = color
        self._powersource = powersource
        self._type = type 
        self._uid = uid
    
    # a getter method, extracts uid from object
    @property
    def uid(self):
        return self._uid
    
    # a setter function, allows uid to be updated after initial object creation
    @uid.setter
    def uid(self, uid):
        self._uid = uid
        
    # gets the brand the car
    @property
    def brand(self):
        return self._brand
    
    # a setter function, allows brand to be updated after initial object creation
    @brand.setter
    def brand(self, brand):
        self._brand = brand

     # gets the color of the car
    @property
    def color(self):
        return self._color
    
    # a setter function, allows color to be updated after initial object creation
    @color.setter
    def color(self, color):
        self._color = color

    # gets the type of the manufacturer or the car
    @property
    def type(self):
        return self._type
    
    # a setter function, allows type to be updated after initial object creation
    @type.setter
    def type(self, type):
        self._type = type
    
    # a powersource getter
    @property
    def powersource(self):
        return self._powersource

    # a setter function to set the car's powersource
    @powersource.setter
    def powersource(self, powersource):
        self._powersource = powersource
    
            
    # output content using str(object) in human readable form, uses getter
    # output content using json dumps, this is ready for API response
    def __str__(self):
        return json.dumps(self.read())

    # CRUD create/add a new record to the table
    # returns self or None on error
    def create(self):
        try:
            # creates a person object from User(db.Model) class, passes initializers
            db.session.add(self)  # add prepares to persist person object to Users table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            db.session.remove()
            return None

    # CRUD read converts self to dictionary
    # returns dictionary
    def read(self):
        return {
            "id": self.id,
            "brand" : self.brand,
            "color" : self.color,
            "powersource" : self.powersource,
            "type" : self.type,
            "uid": self.uid
        }
     
    def update(self, brand="", color="", powersource="", type="", uid=""):
        """only updates values with length"""
        if len(brand) > 0:
            self.brand = brand
        if len(color) > 0:
            self.color = color
        if len(powersource) > 0:
            self.powersource = powersource
        if len(type) > 0:
            self.type = type
        if len(uid) > 0:
            self.uid = uid
        db.session.add(self) # performs update when id exists\n",
        db.session.commit()
        return self

    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None

## Initial Data
- Comment on how these work?
    1. Create All Tables from db Object: 
        - Uses SQLALchemy db.create_all() to initialize rows into sqlite.db 
    2. User Object Constructors: 
        - uses getter and setter methods to initialize user objects.
    3. Try / Except : 
        - this feature is cleaning out bad or replicated data. It ensures that there will be no user with the same id.


### My database data initializer

In [20]:
def initCars():
    with app.app_context():
        """Create database and tables"""
        db.create_all()

        """Tester data for table"""
        c1 = Car(uid='1', brand='Acura', color='gray', powersource='ice', type='suv')
        c2 = Car(uid='2', brand='Hyundai', color='red', powersource='ice', type='sedan') 
        c3 = Car(uid='3', brand='Mazda', color='white', powersource='ice', type='sedan')
        c4 = Car(uid='4', brand='Honda', color='gray', powersource='ice', type='suv')
        c5 = Car(uid='5', brand='Dodge', color='black', powersource='ice', type='suv')
        c6 = Car(uid='6', brand='Toyota', color='white', powersource='ice', type='truck') 
        c7 = Car(uid='7', brand='Hyundai', color='blue', powersource='ice', type='sedan')
        c8 = Car(uid='8', brand='Chevrolet', color='gray', powersource='ice', type='truck')
        c9 = Car(uid='9', brand='Jeep', color='gray', powersource='ice', type='suv')
        c10 = Car(uid='10', brand='Nissan', color='silver', powersource='ice', type='sedan') 
        c11 = Car(uid='11', brand='Lexus', color='black', powersource='ice', type='sedan')
        c12 = Car(uid='12', brand='Kia', color='red', powersource='ice', type='suv')
        c13 = Car(uid='13', brand='Mazda', color='red', powersource='ice', type='truck')
        c14 = Car(uid='14', brand='Ford', color='white', powersource='ice', type='sedan') 
        c15 = Car(uid='15', brand='Kia', color='red', powersource='ice', type='truck')
        c16 = Car(uid='16', brand='Ford', color='gray', powersource='ice', type='suv')
        c17 = Car(uid='17', brand='Jeep', color='red', powersource='ice', type='truck')
        c18 = Car(uid='18', brand='Toyota', color='red', powersource='electric', type='suv') 
        c19 = Car(uid='19', brand='Kia', color='silver', powersource='ice', type='truck')
        c20 = Car(uid='20', brand='Honda', color='white', powersource='ice', type='suv')
        c21 = Car(uid='21', brand='Hyundai', color='white', powersource='ice', type='sedan')
        c22 = Car(uid='22', brand='Chevrolet', color='white', powersource='ice', type='suv') 
        c23 = Car(uid='23', brand='Jeep', color='white', powersource='ice', type='suv')
        c24 = Car(uid='24', brand='BMW', color='gray', powersource='ice', type='sedan')
        c25 = Car(uid='25', brand='Ferrari', color='yellow', powersource='ice', type='sports')
        c26 = Car(uid='26', brand='Tesla', color='red', powersource='electric', type='suv') 
        c27 = Car(uid='27', brand='Tesla', color='blue', powersource='electric', type='suv')
        c28 = Car(uid='28', brand='Ford', color='white', powersource='electric', type='truck')
        c29 = Car(uid='29', brand='Ford', color='blue', powersource='electric', type='truck')
        c30 = Car(uid='30', brand='Audi', color='black', powersource='electric', type='suv') 
        c31 = Car(uid='31', brand='Ferrari', color='red', powersource='electric', type='sports')
        c32 = Car(uid='32', brand='Mercedes', color='silver', powersource='electric', type='sedan')
        c33 = Car(uid='33', brand='Mazda', color='silver', powersource='electric', type='suv')
        c34 = Car(uid='34', brand='Nissan', color='blue', powersource='electric', type='suv') 
        c35 = Car(uid='35', brand='Subaru', color='red', powersource='electric', type='suv')

        cars = [c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13, c14, c15, c16, c17, c18, c19, c20, c21, c22, c23, c24, c25, c26, c27, c28, c29, c30, c31, c32, c33, c34, c35]

        #try:
        #    db.session.query(Car).delete()
        #    db.session.commit()
        #except:
        #    db.session.rollback()

        """Builds sample car/note(s) data"""
        for car in cars:
            try:
                object = car.create()
                print(f"Created new uid {object.uid}")
                
            except:  # error raised if object nit created
                '''fails with bad or duplicate data'''
                print(f"Records exist uid {car.uid}, or error.")
initCars()

OperationalError: (sqlite3.OperationalError) unable to open database file
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [9]:
"""Database Creation and Testing """

# Builds working data for testing
def initUsers():
    with app.app_context():
        """Create database and tables"""
        db.create_all()
        """Tester data for table"""
        u1 = User(name='Thomas Edison', uid='toby', password='123toby', dob=datetime(1847, 2, 11))
        u2 = User(name='Nikola Tesla', uid='niko', password='123niko')
        u3 = User(name='Alexander Graham Bell', uid='lex', password='123lex')
        u4 = User(name='Eli Whitney', uid='whit', password='123whit')
        u5 = User(name='Indiana Jones', uid='indi', dob=datetime(1920, 10, 21))
        u6 = User(name='Marion Ravenwood', uid='raven', dob=datetime(1921, 10, 21))


        users = [u1, u2, u3, u4, u5, u6]

        """Builds sample user/note(s) data"""
        for user in users:
            try:
                '''add user to table'''
                object = user.create()
                print(f"Created new uid {object.uid}")
            except:  # error raised if object nit created
                '''fails with bad or duplicate data'''
                print(f"Records exist uid {user.uid}, or error.")
                
initUsers()

OperationalError: (sqlite3.OperationalError) unable to open database file
(Background on this error at: https://sqlalche.me/e/14/e3q8)

## Check for given Credentials in users table in sqlite.db
> Use of ORM Query object and custom methods to identify user to credentials uid and password

- Comment on purpose of following
    1. User.query.filter_by: 
        - this filters the database by uid and identifies the user whose uid matches the input
    2. user.password: 
        - once the uid is received, the function check-credentials compares that inputted password to the password of the user with the inputted uid. In essence, this is like a security feature. 

In [4]:
# SQLAlchemy extracts single user from database matching User ID
def find_by_uid(uid):
    with app.app_context():
        user = User.query.filter_by(_uid=uid).first()
    return user # returns user object

def find_by_caruid(uid):
    with app.app_context():
        car = Car.query.filter_by(_uid=uid).first()
    return car # returns user object

# Check credentials by finding user and verify password
def check_credentials(uid, password):
    # query email and return user record
    user = find_by_uid(uid)
    if user == None:
        return False
    if (user.is_password(password)):
        return True
    return False

#check_credentials("indi", "123qwerty")

## Create a new User in table in Sqlite.db
> Uses SQLALchemy and custom user.create() method to add row.

- Comment on purpose of following
    1. user.find_by_uid() and try/except:  
        - the find_by_uid() filters the database by uid and identifies the user whose uid matches the input. it then returns that user.
        - the try/except feature is cleaning out bad or replicated data. It ensures that there will be no user with the same id.
    2. user = User(...)
        - this initializes the user objects before date of birth
    3. user.dob and try/except:  
        - user.dob is a placeholder for the birth date of the user. It ensures that users write their date of birth in the right format otherwise an exception occurs and the date of birth is set to the default.
        - the try/except feature is cleaning out bad or replicated data. It ensures that there will be no user with the same id.
    4. user.create() and try/except:  
        - user.create() creates a new user to add the the sqlite table based off of the input it receives from the user.
        - the try/except feature is cleaning out bad or replicated data. It ensures that there will be no user with the same id.

In [20]:
# Inputs, Try/Except, and SQLAlchemy work together to build a valid database object
def create():
    # optimize user time to see if uid exists
    uid = input("Enter your user id:")
    user = find_by_uid(uid)
    try:
        print("Found\n", user.read())
        return
    except:
        pass # keep going
    
    # request value that ensure creating valid object
    name = input("Enter your name:")
    password = input("Enter your password")
    
    # Initialize User object before date
    user = User(name=name, 
                uid=uid, 
                password=password
                )
    
    # create user.dob, fail with today as dob
    dob = input("Enter your date of birth 'YYYY-MM-DD'")
    try:
        user.dob = datetime.strptime(dob, '%Y-%m-%d').date()
    except ValueError:
        user.dob = datetime.today()
        print(f"Invalid date {dob} require YYYY-mm-dd, date defaulted to {user.dob}")
           
    # write object to database
    with app.app_context():
        try:
            object = user.create()
            print("Created\n", object.read())
        except:  # error raised if object not created
            print("Unknown error uid {uid}")
        
create()

Created
 {'id': 7, 'name': 'Vivian Ni', 'uid': 'vivian', 'dob': '05-13-2006', 'age': 16}


### Create and Read for Car database (my database)

In [5]:
def createCar():
    uid = input("Enter your car id:")
    car = find_by_caruid(uid)
    try:
        print("Found\n", car.read())
        return
    except:
        pass # keep going

    # request value that ensure creating valid object
    brand = input("What brand is the car?")
    color = input("What color is the car?")
    powersource = input("What is the car's powersource?")
    type = input("What type of car is it?") 
    
    # Initialize User object before date
    car = Car(brand=brand,
                color=color, 
                powersource=powersource,
                type=type,
                uid=uid
                )
       
    # write object to database
    with app.app_context():
        try:
            object = car.create()
            print("Created\n", object.read())
        except:  # error raised if object not created
            print("Unknown error powersource {uid}")
        
createCar()

Created
 {'id': 36, 'brand': 'tesla', 'color': 'pink', 'powersource': 'ice', 'type': 'sedan', 'uid': '36'}


## Reading users table in sqlite.db
> Uses SQLALchemy query.all method to read data

- Comment on purpose of following
    1. User.query.all
    2. json_ready assignment, google List Comprehension

In [ ]:

# SQLAlchemy extracts all users from database, turns each user into JSON
def read():
    with app.app_context():
        table = User.query.all()
    json_ready = [user.read() for user in table] # "List Comprehensions", for each user add user.read() to list
    return json_ready

read()

In [ ]:
def readCar():
    with app.app_context():
        table = Car.query.all()
    json_ready = [car.read() for car in table] # "List Comprehensions", for each car add car.read() to list
    return json_ready

readCar()

# Hacks

## Update functionality
- First for the Users

In [21]:
def update(uid, new_name):
    with app.app_context():
        user = db.session.query(User).filter_by(_uid=uid).first()
        try:
            print("Found\n", user.read())

            user.update(new_name)
        
            return
        except:
            pass # keep going

uid = input("Enter your user id:")
new_name = input("Enter new user name:")
update(uid, new_name)
    

Found
 {'id': 7, 'name': 'Vivian Ni', 'uid': 'vivian', 'dob': '05-13-2006', 'age': 16}


- for the cars

In [7]:
def updateCar(uid, new_brand, new_color, new_powersource, new_type):
    with app.app_context():
        car = db.session.query(Car).filter_by(_uid=uid).first()
        try:
            print("Found\n", car.read())

            car.update(new_brand, new_color, new_powersource, new_type)
        
            return
        except:
            pass # keep going

uid = input("Enter your car id:")

new_brand = input("Enter new brand:")
new_color = input("Enter new color:")
new_powersource = input("Enter new powersource:")
new_type = input("Enter new type:")

updateCar(uid, new_brand, new_color, new_powersource, new_type)

Found
 {'id': 36, 'brand': 'tesla', 'color': 'pink', 'powersource': 'ice', 'type': 'sedan', 'uid': '36'}


## Delete functionality 
- for the users

In [ ]:
def delete(uid):
    with app.app_context():
        user = db.session.query(User).filter_by(_uid=uid).first()
        try:
            print("Found\n", user.read())

            user.delete()
        
            return
        except:
            pass # keep going

uid = input("Enter your user id:")
delete(uid)

- for the cars

In [9]:
def deleteCar(uid):
    with app.app_context():
        car = db.session.query(Car).filter_by(_uid=uid).first()
        try:
            print("Found\n", car.read())

            car.delete()
        
            return
        except:
            pass # keep going

uid = input("Enter your car uid:")
deleteCar(uid)

Found
 {'id': 36, 'brand': 'tesla', 'color': 'pink', 'powersource': 'electric', 'type': 'sedan', 'uid': '36'}
